# Grand Canyon DEM - Large Dataset Test

#### run "uv pip install rio-vrt" before running this notebook

In [ ]:
from pathlib import Path
import math
import leafmap
import rioxarray
from rasterio.warp import transform_bounds
from jupyter_xarray_tiler.titiler import add_data_array
from rio_vrt import build_vrt

#### FILL IN THE PATH BELOW!

In [ ]:
# INSERT DIRECTORY PATH HERE:
dem_dir = Path("/path/to/local/DEMs/folder")
#I reccomend using the dataset from https://www.sciencebase.gov/catalog/item/62ab6770d34e74f0d80eb3af

In [ ]:
# Load all TIFFs
tif_files = sorted(dem_dir.glob("*.tif"))
print(tif_files)

## Use all the tif's through a VRT w/ Dask
### currently does not work

In [ ]:
# Create VRT (virtual mosaic) and open with lazy loading
vrt_path = dem_dir / "mosaic.vrt"
build_vrt(str(vrt_path), [str(f) for f in tif_files])
dem = rioxarray.open_rasterio(vrt_path, chunks='auto')
dem

In [ ]:
# Calculate map center and zoom from data extent
bounds = dem.rio.bounds()
lon_min, lat_min, lon_max, lat_max = transform_bounds(dem.rio.crs, "EPSG:4326", *bounds)

center = [(lat_min + lat_max) / 2, (lon_min + lon_max) / 2]
max_diff = max(lat_max - lat_min, lon_max - lon_min)
zoom = int(math.floor(math.log2(360 / max_diff))) - 1

# Add to TiTiler
url = await add_data_array(dem, rescale=(0, 1000), colormap_name="terrain")

# Create map centered on data
m = leafmap.Map(center=center, zoom=zoom)
m.add_tile_layer(url=url, name="DEM", attribution="")
m

## Single TIF Test

### Loading a single TIF (without dask) works correctly.

In [ ]:
# Load the first tif into memory without dask
single_tif = rioxarray.open_rasterio(tif_files[0])

# Calculate map bounds
bounds = single_tif.rio.bounds()
lon_min, lat_min, lon_max, lat_max = transform_bounds(single_tif.rio.crs, "EPSG:4326", *bounds)
center = [(lat_min + lat_max) / 2, (lon_min + lon_max) / 2]
zoom = int(math.floor(math.log2(360 / max(lat_max - lat_min, lon_max - lon_min)))) - 1

# add to TiTiler
url = await add_data_array(single_tif, rescale=(0, 1000), colormap_name="terrain")
m = leafmap.Map(center=center, zoom=zoom)
m.add_tile_layer(url=url, name="Single TIF", attribution="")
m